## Trained model and optimal hyperparameters

We have provided trained model and hyperparameters of SphereNet on QM9 and MD17 [here](https://github.com/divelab/DIG_storage/tree/main/3dgraph).

## Example of 3D Graph

Here we provide the example code for SphereNet on QM93D and MD17 datasets. You can easily replace SphereNet with SchNet, DimeNetPP, and ComENet by changing model name and model parameters.

In [2]:
import torch
from dig.threedgraph.dataset import QM93D
from dig.threedgraph.dataset import MD17
from dig.threedgraph.method import SphereNet #SchNet, DimeNetPP, ComENet
from dig.threedgraph.method import run
from dig.threedgraph.evaluation import ThreeDEvaluator

In [3]:
device = torch.device('cuda:0') if torch.cuda.is_available() else torch.device("cpu")
device

device(type='cuda', index=0)

## 1. QM9

**QM9 dataset includes geometric, energetic, electronic, and thermodynamic properties for 134k stable small organic molecules.**

There are 12 properties. Here are the units for each property.

|                          | mu     | alpha  | homo | lumo | gap  | r2    | zpve | U0   | U    | H    | G    | Cv     | std. MAE |
| ------------------------ | ------ | ------ | ---- | ---- | ---- | ----- | ---- | ---- | ---- | ---- | ---- | ------ | -------- |
| Unit in the dataset      | D      | a_0^3  | eV   | eV   | eV   | a_0^2 | eV   | eV   | eV   | eV   | eV   | cal / mol K  |    |
| Unit of the reported MAE | D      | a_0^3  | meV  | meV  | meV  | a_0^2 | meV  | meV  | meV  | meV  | meV  | cal / mol K  | %  |

**We trained a separate model for each target** except for _gap_, which was predicted by taking _lumo-homo_.

**The default hyperparameters for QM9 are:** 
    &ensp; energy_and_force=False, cutoff=5.0, num_layers=4, hidden_channels=128, out_channels=1, int_emb_size=64,  
    &ensp; basis_emb_size_dist=8, basis_emb_size_angle=8, basis_emb_size_torsion=8, out_emb_channels=256,  
    &ensp; num_spherical=3, num_radial=6, envelope_exponent=5,  
    &ensp; num_before_skip=1, num_after_skip=2, num_output_layers=3,  
    &ensp; epochs=1000, batch_size=32, vt_batch_size=32, lr=0.0005, lr_decay_factor=0.5, lr_decay_step_size=100.




### Trained model

We provide trained models to obtain optimal results.
The models are in https://github.com/divelab/DIG_storage/tree/main/3dgraph/qm9.

The results using the trained models are listed in the table. 

|            | mu     | alpha  | homo | lumo | gap  | r2    | zpve | U0   | U    | H    | G    | Cv     | std. MAE |
| ---------- | ------ | ------ | ---- | ---- | ---- | ----- | ---- | ---- | ---- | ---- | ---- | ------ | -------- |
| MAE | 0.0245 | 0.0449 | 22.8 | 19.6 | 31.1 | 0.268 | 1.26 | 6.31 | 6.36 | 6.33 | 7.78 | 0.024  | 0.92     |

#### Loading dataset

In [3]:
dataset = QM93D(root='dataset/')
target = 'U0' # choose from: mu, alpha, homo, lumo, r2, zpve, U0, U, H, G, Cv
dataset.data.y = dataset.data[target]

split_idx = dataset.get_idx_split(len(dataset.data.y), train_size=110000, valid_size=10000, seed=42)

train_dataset, valid_dataset, test_dataset = dataset[split_idx['train']], dataset[split_idx['valid']], dataset[split_idx['test']]
print('train, validaion, test:', len(train_dataset), len(valid_dataset), len(test_dataset))

train, validaion, test: 110000 10000 10831


#### Loading model, loss and evaluation function

The evaluation metric is mean absolute error (MAE).

In [4]:
model = SphereNet(energy_and_force=False, cutoff=5.0, num_layers=4, 
        hidden_channels=128, out_channels=1, int_emb_size=64, 
        basis_emb_size_dist=8, basis_emb_size_angle=8, basis_emb_size_torsion=8, out_emb_channels=256, 
        num_spherical=3, num_radial=6, envelope_exponent=5, 
        num_before_skip=1, num_after_skip=2, num_output_layers=3, use_node_features=True
        )
loss_func = torch.nn.L1Loss()
evaluation = ThreeDEvaluator()

#### Training

In [5]:
run3d = run()
run3d.run(device, train_dataset, valid_dataset, test_dataset, model, loss_func, evaluation, epochs=20, batch_size=32, vt_batch_size=32, lr=0.0005, lr_decay_factor=0.5, lr_decay_step_size=15)

#Params: 1890118

=====Epoch 1

Training...


100%|██████████| 3438/3438 [07:22<00:00,  7.77it/s]



Evaluating...



100%|██████████| 313/313 [00:20<00:00, 15.30it/s]



Testing...


...



Evaluating...



100%|██████████| 313/313 [00:10<00:00, 29.59it/s]



Testing...



100%|██████████| 339/339 [00:11<00:00, 29.71it/s]



{'Train': 0.04962607438894397, 'Validation': 0.04090351238846779, 'Test': 0.040894996374845505}
Best validation MAE so far: 0.03810606151819229
Test MAE when got best validation result: 0.03703922778367996


## 2. MD17

**MD17 is a collection of eight molecular dynamics simulations for small organic molecules.**

**The units for energy and force are kcal / mol and kcal / mol A.**

**We trained a separate model for each molecule.** 

**The default hyperparameters for MD17 are:** 
    &ensp; energy_and_force=True, cutoff=5.0, num_layers=4, hidden_channels=128, out_channels=1, int_emb_size=64,  
    &ensp; basis_emb_size_dist=8, basis_emb_size_angle=8, basis_emb_size_torsion=8, out_emb_channels=256,  
    &ensp; num_spherical=3, num_radial=6, envelope_exponent=5,  
    &ensp; num_before_skip=1, num_after_skip=2, num_output_layers=3,  
    &ensp; epochs=1000, batch_size=1, vt_batch_size=32, lr=0.0005, lr_decay_factor=0.5, lr_decay_step_size=200.

### Trained model

We provide trained models to obtain optimal results.
The models are in https://github.com/divelab/DIG_storage/tree/main/3dgraph/md17.

The results using the trained models are listed in the table. 

|         | aspirin | benzene2017 | ethanol | malonaldehyde | naphthalene | salicylic | toluene | uracil |
| ---------------- | ------- | ----------- | ------- | ------------- | ----------- | --------- | ------- | ------ |
| force MAE    |0.375|0.181|0.187|0.273|0.139|0.284|0.142|0.245|

#### Loading dataset

In [7]:
dataset_md17 = MD17(root='dataset/', name='aspirin')

split_idx_md17 = dataset_md17.get_idx_split(len(dataset_md17.data.y), train_size=1000, valid_size=1000, seed=42)

train_dataset_md17, valid_dataset_md17, test_dataset_md17 = dataset_md17[split_idx_md17['train']], dataset_md17[split_idx_md17['valid']], dataset_md17[split_idx_md17['test']]
print('train, validaion, test:', len(train_dataset_md17), len(valid_dataset_md17), len(test_dataset_md17))

train, validaion, test: 1000 1000 209762


#### Loading model, loss and evaluation function

We predict energy and take the negative of the derivative of the energy with respect to the atomic positions as predicted forces.

The evaluation metric is mean absolute error (MAE).

In [8]:
model_md17 = SphereNet(energy_and_force=True, cutoff=5.0, num_layers=4, 
        hidden_channels=128, out_channels=1, int_emb_size=64, 
        basis_emb_size_dist=8, basis_emb_size_angle=8, basis_emb_size_torsion=8, out_emb_channels=256, 
        num_spherical=3, num_radial=6, envelope_exponent=5, 
        num_before_skip=1, num_after_skip=2, num_output_layers=3 
        )
loss_func_md17 = torch.nn.L1Loss()
evaluation_md17 = ThreeDEvaluator()

#### Training

In [9]:
run3d_md17 = run()
run3d_md17.run(device, train_dataset_md17, valid_dataset_md17, test_dataset_md17, model_md17, loss_func_md17, evaluation_md17, epochs=5, batch_size=1, vt_batch_size=64, lr=0.0005, lr_decay_factor=0.5, lr_decay_step_size=200, energy_and_force=True)

#Params: 1890118

=====Epoch 1

Training...


/data1/limei/anaconda3/envs/graph/lib/python3.8/site-packages/torch_geometric/deprecation.py:13: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)
100%|██████████| 1000/1000 [03:20<00:00,  4.99it/s]



Evaluating...



100%|██████████| 16/16 [00:03<00:00,  5.06it/s]

{'Energy MAE': 21076.162109375, 'Force MAE': 88.1651611328125}


Testing...


...



Evaluating...



100%|██████████| 16/16 [00:03<00:00,  5.20it/s]

{'Energy MAE': 19717.771484375, 'Force MAE': 30.181947708129883}


Testing...



100%|██████████| 3278/3278 [10:41<00:00,  5.11it/s]

{'Energy MAE': 19717.19140625, 'Force MAE': 30.4222354888916}

{'Train': 12309.488090698242, 'Validation': 22735.96625518799, 'Test': 22759.41495513916}
Best validation MAE so far: 15918.326904296875
Test MAE when got best validation result: 15933.84245300293
